In [43]:
from clase_disco import Disco
import sqlite3

DISCOS_DB = "discos.db"

def get_db():
    conn = sqlite3.connect(DISCOS_DB)
    return conn


def create_tables():
    tables = [
        """CREATE TABLE IF NOT EXISTS discos(
                id INTEGER PRIMARY KEY,
                titulo TEXT NOT NULL,
                artista TEXT NOT NULL,
                precio REAL NOT NULL,
                fecha INTEGER NOT NULL,
                idioma TEXT NOT NULL,
                n_canciones INTEGER NOT NULL,
                genero TEXT NOT NULL
            )
            """
    ]
    db = get_db()
    cursor = db.cursor()
    for table in tables:
        cursor.execute(table)

        
def insert_disco(id, titulo, artista, precio, fecha, idioma, n_canciones, genero):
    db = get_db()
    cursor = db.cursor()
    statement = "INSERT INTO discos (id, titulo, artista, precio, fecha, idioma, n_canciones, genero) \
    VALUES ( ?, ?, ?, ? ,?, ?, ?, ?)"
    cursor.execute(statement, [id, titulo, artista, precio, fecha, idioma, n_canciones, genero])
    db.commit()
    return True


def update_disco(id, titulo, artista, precio, fecha, idioma, n_canciones, genero):
    db = get_db()
    cursor = db.cursor()
    statement = "UPDATE discos SET titulo = ?, artista = ?, precio= ?, fecha= ?, idioma= ?, n_canciones= ?, \
    genero= ?"
    cursor.execute(statement, [titulo, artista, precio, fecha, idioma, n_canciones, genero])
    db.commit()
    return True


def delete_disco(id):
    db = get_db()
    cursor = db.cursor()
    statement = "DELETE FROM discos WHERE id = ?"
    cursor.execute(statement, [id])
    db.commit()
    return True


def get_by_id(id):
    db = get_db()
    cursor = db.cursor()
    statement = "SELECT id, titulo, artista, precio, fecha, idioma, n_canciones, genero FROM discos WHERE id = ?"
    cursor.execute(statement, [id])
    single_disco = cursor.fetchone()
    id = single_disco[0]
    titulo = single_disco[1]
    artista = single_disco[2]
    precio = single_disco[3]
    fecha = single_disco[4]
    idioma = single_disco[5]
    n_canciones = single_disco[6]
    genero = single_disco[7]
    disco = Disco(id, titulo, artista, precio, fecha, idioma, n_canciones, genero)
    return disco.describe_details()


def get_discos():
    db = get_db()
    cursor = db.cursor()
    query = "SELECT id, titulo, artista, precio, fecha, idioma, n_canciones, genero FROM discos"
    cursor.execute(query)
    discos_list = cursor.fetchall()
    list_of_discos=[]
    for disco in discos_list:
        id = disco[0]
        titulo = disco[1]
        artista = disco[2]
        precio = disco[3]
        fecha = disco[4]
        idioma = disco[5]
        n_canciones = disco[6]
        genero = disco[7]
        disco_to_add = Disco(id, titulo, artista, precio, fecha, idioma, n_canciones, genero)
        list_of_discos.append(disco_to_add)
    return list_of_discos


In [44]:
pip install spotify

Note: you may need to restart the kernel to use updated packages.


In [45]:
pip install spotipy --upgrade

Note: you may need to restart the kernel to use updated packages.


In [46]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [47]:
def consultar_canciones(nombre_album):
    clientID = '451920e7f5404190bb01798fb5ca0374'
    clientSecret = '196facfb0eab4ceba79aeda2e8cbf98f'
    
    client_credential_manager = SpotifyClientCredentials(client_id=clientID, client_secret=clientSecret)
    sp = spotipy.Spotify(client_credentials_manager=client_credential_manager)
    
    resultados = sp.search(q=f'album:"{nombre_album}"', type='album')
    
    if 'albums' in resultados:
        items = resultados['albums'].get('items', [])
        if items:
            album_id = items[0]['id']
            dic = sp.album_tracks(f'spotify:album:{album_id}')
            list_info_canciones = dic.get('items', [])
            if list_info_canciones:
                lista = [cancion['name'] for cancion in list_info_canciones]
                return lista
    else:
        return f"No se encontraron resultados para el álbum '{nombre_album}'"

In [ ]:
from flask import Flask, jsonify, request
app = Flask(__name__)


@app.route('/disco', methods=["GET"])
def get_disco():
    discos = get_discos()
    disco_list=[]
    for disco in discos:
        elem = disco.describe()
        disco_list.append(elem)
    return jsonify(disco_list)

@app.route("/disco/create", methods=["POST"])
def insertar_disco():
    disco_details = request.get_json()
    id = disco_details["id"]
    titulo = disco_details["titulo"]
    artista =disco_details["artista"]
    precio = disco_details["precio"]
    fecha = disco_details["fecha"]
    idioma = disco_details["idioma"]
    n_canciones = disco_details["n_canciones"]
    genero = disco_details ["genero"]
    result = insert_disco(id,titulo,artista,precio,fecha,idioma, n_canciones, genero)
    return jsonify(result)


@app.route("/disco/modify", methods=["PUT"])
def update_disco_db():
    disco_details = request.get_json()
    id = disco_details["id"]
    titulo = disco_details["titulo"]
    artista =disco_details["artista"]
    precio = disco_details["precio"]
    fecha = disco_details["fecha"]
    idioma = disco_details["idioma"]
    n_canciones = disco_details["n_canciones"]
    genero = disco_details["genero"]
    result = update_disco(id,titulo,artista,precio,fecha,idioma, n_canciones, genero)
    return jsonify(result)


@app.route("/disco/eliminate/<id>", methods=["DELETE"])
def delete_disco_db(id):
    result = delete_disco(id)
    return jsonify(result)


@app.route("/disco/<id>", methods=["GET"])
def get_disco_by_id_db(id):
    disco = get_by_id(id)
    return jsonify(disco)

    
@app.route("/disco/songs/<titulo>", methods=["GET"])
def get_disco_by_titulo_songs(titulo):
    canciones = consultar_canciones(titulo)
    return jsonify(canciones)

create_tables()

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Nov/2023 16:19:07] "GET /disco/songs/rfjnrjfwdñ HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2023 16:19:13] "GET /disco/songs/cupido HTTP/1.1" 200 -
